In [ ]:
from argparse import ArgumentParser
from generators_binary import PatchGenerator, PatchSequence
from keras.callbacks import Callback
from keras.models import load_model
from keras.optimizers import Adam, RMSprop
from keras.utils import multi_gpu_model
from networks_binary import (create_initial_model,
                      create_second_model,
                      create_squeezenet3d_model,
                      create_squeezenet3d_model2
                     )
from skimage.transform import rotate
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight

# machine learning / deep learning
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)

import keras.backend as K
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys, os, warnings

DATADIR = '/projects/0/ismi2018/FINALPROJECTS/BREAST_3D_ULTRASOUND/shareWithStudents'

NETWORKS = {
    'initial': create_initial_model,
    'second': create_second_model,
    'squeezenet3d': create_squeezenet3d_model,
    'squeezenet3d2': create_squeezenet3d_model2
}

LOSS_FUNCTION = 'categorical_crossentropy'

OPTIMIZERS = {
    'adam': Adam,
    'rmsprop': RMSprop,
}


class MultiGPUCheckpoint(Callback):

    def __init__(self, filename, verbose=0):
        super().__init__()
        self.filename = filename
        self.verbose = verbose
        self.val_loss = []

    def on_epoch_end(self, epoch, logs=None):
        if not self.val_loss:
            self.model.layers[-2].save(self.filename)
        elif logs['val_loss'] < min(self.val_loss):
            if self.verbose > 0:
                print('Saving to {}'.format(self.filename))
            self.model.layers[-2].save(self.filename)
        self.val_loss.append(logs['val_loss'])


class Accuracies(Callback):

    def __init__(self, valid_seq, step = 0):
        super().__init__()
        self.valid_seq = valid_seq
        self.label_accuracies = []
        self.step = step

    def on_epoch_end(self, epoch, logs=None):
        
        y_pred = self.model.predict_generator(self.valid_seq,
                                              workers=4,
                                              use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = self.valid_seq.get_all_labels()
        
        if self.step > 0:
            y_true = y_true[y_true != 21]
            y_true[y_true == 2] = 1
            y_true[y_true == 20] = 0
        else:
            y_true[y_true == 2] = 1
            y_true[y_true == 20] = 1
            y_true[y_true == 21] = 0
        
        
        cm = confusion_matrix(y_true, y_pred)
        ps = cm.diagonal() / cm.sum(axis=1)
        self.label_accuracies.append(ps)


def create_model(network, optimizer, drop_rate, multi_gpu):
    orig_model = NETWORKS[network](drop_rate=drop_rate)
    if multi_gpu:
        parallel_model = multi_gpu_model(orig_model)
        parallel_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION,
                               metrics=['accuracy'])
    else:
        orig_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION,
                           metrics=['accuracy'])
        parallel_model = None
    return orig_model, parallel_model


def create_optimizer(name, lr, decay):
    return OPTIMIZERS[name](lr=lr, decay=decay)


def make_augmentation_func(aug, aug_hflip, aug_vflip, aug_rotate,
                           aug_brightness):
    if not aug:
        return None

    def augf(img):
        if np.random.random() > aug and aug_hflip:
            img = np.flip(img, axis = 1)
        if np.random.random() > aug and aug_vflip:
            img = np.flip(img, axis = 0)
        if np.random.random() > aug and aug_rotate:
            tmp = np.squeeze(img)
            angle = np.random.uniform(0, aug_rotate)
            tmp = rotate(tmp, angle)
            img = np.expand_dims(tmp, -1)
        if np.random.random() > aug and aug_brightness:
            up_delta = 1. - img.max()
            down_delta = img.min()
            delta = min(up_delta, down_delta)
            img = img + np.random.uniform(-delta, delta)
        return img[15:55, 15:55, ...]

    return augf


def make_generators(csv, train_patients, validation_patients, batch_size, step,
                    augf):
    train_csv = csv.loc[csv['patientID'].isin(train_patients), :]
    valid_csv = csv.loc[csv['patientID'].isin(validation_patients), :]

    train_gen = PatchGenerator(
        input_dir=DATADIR,
        dataframe=train_csv,
        batch_size=batch_size,
        step=step,
        augmentation_fn=augf
    )

    valid_seq = PatchSequence(
        input_dir=DATADIR,
        dataframe=valid_csv,
        batch_size=batch_size,
        step=step
    )

    return train_gen, valid_seq


def train_model_1st():
    csv = pd.read_csv(os.path.join(DATADIR, 'trainingSet.csv'), dtype=str)

    # Create patient K-folder
    unique_patients = csv.patientID.unique()
    kf = KFold(5, shuffle=True, random_state=42)
    folds = kf.split(unique_patients)

    # Make augmentation function
    augf = make_augmentation_func(0.5, True, True, True, True)
    
    # -----------------------------------------------------------------------
        
    accuracies_first = []
    # first step training
    for i, (train_idxs, val_idxs) in enumerate(folds, start=1):
        K.clear_session()
        print('Fold {}'.format(i))

        train_patients = unique_patients[train_idxs]
        val_patients = unique_patients[val_idxs]

        train_gen, valid_seq = make_generators(csv,
                                               train_patients,
                                               val_patients,
                                               6,
                                               0,
                                               augf)
        
        optimizer = create_optimizer('adam', 1e-5, 1e-6)
        orig_net, parallel_net = create_model('squeezenet3d', optimizer,
                                              0.5,
                                              True)

        save_filename = './results_1st/{}_fold_{}.h5'.format('1st-step', i)
        
        if True:
            cp = MultiGPUCheckpoint(save_filename, verbose=1)
        else:
            cp = ModelCheckpoint(save_filename, save_best_only=True, verbose=1,
                                 monitor='val_acc')
        ps = Accuracies(valid_seq, 0)

        train_model = parallel_net or orig_net
                
        results = train_model.fit_generator(train_gen,
                                            steps_per_epoch=len(train_gen),
                                            validation_data=valid_seq,
                                            epochs=250,
                                            workers=4,
                                            use_multiprocessing=True,
                                            callbacks=[cp, ps],
                                            verbose=1)

        h = results.history
        plt.figure()
        plt.plot(h['loss'])
        plt.plot(h['acc'])
        plt.plot(h['val_loss'])
        plt.plot(h['val_acc'])
        plt.legend(['loss', 'acc', 'val_loss', 'val_acc'])
        plt.savefig('{}.traininglog.png'.format(save_filename))

        y_true = valid_seq.get_all_labels()
        y_true[y_true == 2] = 1
        y_true[y_true == 20] = 1
        y_true[y_true == 21] = 0
        best_net = load_model(save_filename)
        y_pred = best_net.predict_generator(valid_seq,
                                            workers=4,
                                            use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        cm = confusion_matrix(y_true, y_pred)

        plt.figure()
        sns.heatmap(cm, annot=True)
        plt.savefig('{}.confusionmatrix.png'.format(save_filename))

        precs = np.array(ps.label_accuracies)

        plt.figure()
        for i in range(precs.shape[1]):
            plt.plot(precs[:, i])
        plt.legend(['0', '1'])
        plt.savefig('{}.accuracies.png'.format(save_filename))

        accuracies_first.append(max(h['val_acc']))

    with open('{}_score.txt'.format('2step-1'), 'w') as f:
        print('Mean accuracy 1st step: {:.4f}\n'.format(np.mean(accuracies_first)), file=f)
        

def train_model_2nd():
    csv = pd.read_csv(os.path.join(DATADIR, 'trainingSet.csv'), dtype=str)

    # Create patient K-folder
    unique_patients = csv.patientID.unique()
    kf = KFold(5, shuffle=True, random_state=42)
    folds = kf.split(unique_patients)

    # Make augmentation function
    augf = make_augmentation_func(0.5, True, True, True, True)
                
    # -------------------------------------------------------------------------
    accuracies_second = []
    # second step training
    for i, (train_idxs, val_idxs) in enumerate(folds, start=1):
        K.clear_session()
        print('Fold {}'.format(i))

        train_patients = unique_patients[train_idxs]
        val_patients = unique_patients[val_idxs]

        train_gen, valid_seq = make_generators(csv,
                                               train_patients,
                                               val_patients,
                                               6,
                                               1,
                                               augf)
        
        optimizer = create_optimizer('adam', 1e-5, 1e-6)
        orig_net, parallel_net = create_model('squeezenet3d', optimizer,
                                              0.5,
                                              True)

        save_filename = './results_2nd/{}_fold_{}.h5'.format('2nd-step', i)
        if True:
            cp = MultiGPUCheckpoint(save_filename, verbose=1)
        else:
            cp = ModelCheckpoint(save_filename, save_best_only=True, verbose=1,
                                 monitor='val_acc')
        ps = Accuracies(valid_seq, 1)

        train_model = parallel_net or orig_net
                
        results = train_model.fit_generator(train_gen,
                                            steps_per_epoch=len(train_gen),
                                            validation_data=valid_seq,
                                            epochs=250,
                                            workers=4,
                                            use_multiprocessing=True,
                                            callbacks=[cp, ps],
                                            verbose=1)

        h = results.history
        plt.figure()
        plt.plot(h['loss'])
        plt.plot(h['acc'])
        plt.plot(h['val_loss'])
        plt.plot(h['val_acc'])
        plt.legend(['loss', 'acc', 'val_loss', 'val_acc'])
        plt.savefig('{}.traininglog.png'.format(save_filename))

        y_true = valid_seq.get_all_labels()
        y_true = y_true[y_true != 21]
        y_true[y_true == 2] = 1
        y_true[y_true == 20] = 0
        best_net = load_model(save_filename)
        y_pred = best_net.predict_generator(valid_seq,
                                            workers=4,
                                            use_multiprocessing=True)
        y_pred = np.argmax(y_pred, axis=1)
        cm = confusion_matrix(y_true, y_pred)

        plt.figure()
        sns.heatmap(cm, annot=True)
        plt.savefig('{}.confusionmatrix.png'.format(save_filename))

        precs = np.array(ps.label_accuracies)

        plt.figure()
        for i in range(precs.shape[1]):
            plt.plot(precs[:, i])
        plt.legend(['0', '1'])
        plt.savefig('{}.accuracies.png'.format(save_filename))

        accuracies_second.append(max(h['val_acc']))
    
    with open('{}_score.txt'.format('2step-2'), 'w') as f:
        print('Mean accuracy 1st step: {:.4f}\n'.format(np.mean(accuracies_second)), file=f)

Using TensorFlow backend.


In [ ]:
train_model_1st()
#train_model_2nd()

Fold 1
PatchGenerator detected: 239 patch samples.
- Cysts: 71 cases
- Tumors: 130 cases
- Fibroadenoma: 38 cases
PatchSequence detected: 59 patch samples.
- Cysts: 18 cases
- Tumors: 36 cases
- Fibroadenoma: 5 cases


/home/ruc0027/.local/lib/python3.5/site-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/250
39/39 [==============================] - 30s 773ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6924 - val_acc: 0.6949
Epoch 2/250
39/39 [==============================] - 22s 574ms/step - loss: 0.6930 - acc: 0.5000 - val_loss: 0.6920 - val_acc: 0.6949
Saving to ./results_1st/1st-step_fold_1.h5
Epoch 3/250
39/39 [==============================] - 23s 582ms/step - loss: 0.6929 - acc: 0.5000 - val_loss: 0.6917 - val_acc: 0.6949
Saving to ./results_1st/1st-step_fold_1.h5
Epoch 4/250
39/39 [==============================] - 23s 578ms/step - loss: 0.6926 - acc: 0.5000 - val_loss: 0.6902 - val_acc: 0.6949
Saving to ./results_1st/1st-step_fold_1.h5
Epoch 5/250
39/39 [==============================] - 23s 582ms/step - loss: 0.6923 - acc: 0.5043 - val_loss: 0.6897 - val_acc: 0.6949
Saving to ./results_1st/1st-step_fold_1.h5
Epoch 6/250
39/39 [==============================] - 23s 577ms/step - loss: 0.6907 - acc: 0.5214 - val_loss: 0.6873 - val_acc: 0.6610
Saving to ./results_1st/

Epoch 54/250
39/39 [==============================] - 22s 573ms/step - loss: 0.5527 - acc: 0.7179 - val_loss: 0.5653 - val_acc: 0.7458
Epoch 55/250
39/39 [==============================] - 22s 567ms/step - loss: 0.4703 - acc: 0.7821 - val_loss: 0.6180 - val_acc: 0.6102
Epoch 56/250
39/39 [==============================] - 23s 578ms/step - loss: 0.5366 - acc: 0.7607 - val_loss: 0.5291 - val_acc: 0.7458
Epoch 57/250
39/39 [==============================] - 22s 576ms/step - loss: 0.5112 - acc: 0.7479 - val_loss: 0.5176 - val_acc: 0.6949
Saving to ./results_1st/1st-step_fold_1.h5
Epoch 58/250
39/39 [==============================] - 22s 571ms/step - loss: 0.4988 - acc: 0.7521 - val_loss: 0.5170 - val_acc: 0.6949
Saving to ./results_1st/1st-step_fold_1.h5
Epoch 59/250
39/39 [==============================] - 22s 576ms/step - loss: 0.4959 - acc: 0.7949 - val_loss: 0.5190 - val_acc: 0.7458
Epoch 60/250
39/39 [==============================] - 22s 572ms/step - loss: 0.4978 - acc: 0.7778 - val_

39/39 [==============================] - 22s 576ms/step - loss: 0.4510 - acc: 0.8034 - val_loss: 0.4598 - val_acc: 0.8475
Epoch 171/250
39/39 [==============================] - 22s 574ms/step - loss: 0.4145 - acc: 0.8205 - val_loss: 0.4691 - val_acc: 0.7797
Epoch 172/250
39/39 [==============================] - 22s 572ms/step - loss: 0.4402 - acc: 0.8248 - val_loss: 0.4377 - val_acc: 0.8644
Epoch 173/250
39/39 [==============================] - 23s 579ms/step - loss: 0.5053 - acc: 0.7735 - val_loss: 0.4330 - val_acc: 0.8814
Epoch 174/250
39/39 [==============================] - 23s 581ms/step - loss: 0.4796 - acc: 0.7479 - val_loss: 0.4435 - val_acc: 0.8814
Epoch 175/250
39/39 [==============================] - 22s 571ms/step - loss: 0.4316 - acc: 0.8077 - val_loss: 0.4529 - val_acc: 0.8644
Epoch 176/250
39/39 [==============================] - 22s 560ms/step - loss: 0.4102 - acc: 0.8376 - val_loss: 0.4197 - val_acc: 0.8814
Epoch 177/250
39/39 [==============================] - 22s 569

...

Fold 2
PatchGenerator detected: 241 patch samples.
- Cysts: 73 cases
- Tumors: 136 cases
- Fibroadenoma: 32 cases
PatchSequence detected: 57 patch samples.
- Cysts: 16 cases
- Tumors: 30 cases
- Fibroadenoma: 11 cases
Epoch 1/250
40/40 [==============================] - 30s 756ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6933 - val_acc: 0.2807
Epoch 2/250
40/40 [==============================] - 22s 547ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6933 - val_acc: 0.2807
Epoch 3/250
40/40 [==============================] - 22s 560ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6936 - val_acc: 0.2807
Epoch 4/250
40/40 [==============================] - 22s 562ms/step - loss: 0.6930 - acc: 0.5000 - val_loss: 0.6939 - val_acc: 0.2807
Epoch 5/250
40/40 [==============================] - 22s 558ms/step - loss: 0.6930 - acc: 0.5000 - val_loss: 0.6943 - val_acc: 0.2807
Epoch 6/250
40/40 [==============================] - 23s 572ms/step - loss: 0.6928 - acc: 0.5000 - val_loss: 0.6

40/40 [==============================] - 22s 562ms/step - loss: 0.4995 - acc: 0.7375 - val_loss: 0.3967 - val_acc: 0.8596
Epoch 116/250
40/40 [==============================] - 22s 562ms/step - loss: 0.4620 - acc: 0.8000 - val_loss: 0.4286 - val_acc: 0.8421
Epoch 117/250
40/40 [==============================] - 22s 545ms/step - loss: 0.4823 - acc: 0.7708 - val_loss: 0.4576 - val_acc: 0.8246
Epoch 118/250
40/40 [==============================] - 22s 556ms/step - loss: 0.4280 - acc: 0.8125 - val_loss: 0.4264 - val_acc: 0.8421
Epoch 119/250
40/40 [==============================] - 22s 550ms/step - loss: 0.4536 - acc: 0.7875 - val_loss: 0.4637 - val_acc: 0.8246
Epoch 120/250
40/40 [==============================] - 22s 554ms/step - loss: 0.4916 - acc: 0.7542 - val_loss: 0.4222 - val_acc: 0.8596
Epoch 121/250
40/40 [==============================] - 23s 564ms/step - loss: 0.4376 - acc: 0.8000 - val_loss: 0.4807 - val_acc: 0.8070
Epoch 122/250
40/40 [==============================] - 22s 552

Epoch 175/250
40/40 [==============================] - 22s 559ms/step - loss: 0.4559 - acc: 0.7667 - val_loss: 0.4191 - val_acc: 0.8421
Epoch 176/250
40/40 [==============================] - 22s 554ms/step - loss: 0.3999 - acc: 0.8375 - val_loss: 0.4384 - val_acc: 0.8421
Epoch 177/250
40/40 [==============================] - 22s 562ms/step - loss: 0.4052 - acc: 0.8375 - val_loss: 0.4453 - val_acc: 0.8246
Epoch 178/250
40/40 [==============================] - 23s 570ms/step - loss: 0.4733 - acc: 0.7917 - val_loss: 0.4376 - val_acc: 0.8246
Epoch 179/250
40/40 [==============================] - 22s 551ms/step - loss: 0.3731 - acc: 0.8292 - val_loss: 0.3921 - val_acc: 0.8596
Epoch 180/250
40/40 [==============================] - 22s 560ms/step - loss: 0.4270 - acc: 0.7875 - val_loss: 0.4604 - val_acc: 0.8070
Epoch 181/250
40/40 [==============================] - 23s 567ms/step - loss: 0.4263 - acc: 0.8208 - val_loss: 0.4079 - val_acc: 0.8596
Epoch 182/250
40/40 [===========================

Epoch 235/250
40/40 [==============================] - 22s 562ms/step - loss: 0.3671 - acc: 0.8583 - val_loss: 0.4927 - val_acc: 0.7895
Epoch 236/250
40/40 [==============================] - 22s 554ms/step - loss: 0.4167 - acc: 0.8250 - val_loss: 0.3684 - val_acc: 0.8421
Epoch 237/250
40/40 [==============================] - 22s 556ms/step - loss: 0.3730 - acc: 0.8250 - val_loss: 0.4779 - val_acc: 0.7895
Epoch 238/250
40/40 [==============================] - 22s 550ms/step - loss: 0.3641 - acc: 0.8458 - val_loss: 0.3947 - val_acc: 0.8246
Epoch 239/250
40/40 [==============================] - 22s 553ms/step - loss: 0.3598 - acc: 0.8750 - val_loss: 0.4693 - val_acc: 0.7895
Epoch 240/250
40/40 [==============================] - 22s 556ms/step - loss: 0.3772 - acc: 0.8375 - val_loss: 0.4424 - val_acc: 0.8246
Epoch 241/250
40/40 [==============================] - 22s 558ms/step - loss: 0.3965 - acc: 0.8292 - val_loss: 0.3939 - val_acc: 0.8596
Epoch 242/250
40/40 [===========================

38/38 [==============================] - 22s 586ms/step - loss: 0.3798 - acc: 0.8728 - val_loss: 0.5296 - val_acc: 0.7391
Epoch 99/250
38/38 [==============================] - 22s 583ms/step - loss: 0.4135 - acc: 0.8421 - val_loss: 0.5044 - val_acc: 0.7391
Epoch 100/250
38/38 [==============================] - 22s 590ms/step - loss: 0.3968 - acc: 0.8202 - val_loss: 0.5364 - val_acc: 0.7101
Epoch 101/250
38/38 [==============================] - 22s 569ms/step - loss: 0.4909 - acc: 0.7719 - val_loss: 0.5489 - val_acc: 0.6667
Epoch 102/250
38/38 [==============================] - 22s 589ms/step - loss: 0.3133 - acc: 0.8904 - val_loss: 0.5426 - val_acc: 0.6957
Epoch 103/250
38/38 [==============================] - 22s 579ms/step - loss: 0.3774 - acc: 0.8377 - val_loss: 0.5047 - val_acc: 0.7391
Epoch 104/250
38/38 [==============================] - 22s 576ms/step - loss: 0.3529 - acc: 0.8465 - val_loss: 0.5458 - val_acc: 0.6957
Epoch 105/250
38/38 [==============================] - 22s 576m

In [ ]:
train_model_2nd()